In [1]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import baseModel as bm
from model import utils as utils
from model import biasMitigation as mit
from model import transferLearning as tl

2022-11-23 19:54:54.377108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 19:54:54.549218: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 19:54:54.549240: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 19:54:54.587381: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 19:54:55.480965: W tensorflow/stream_executor/platform/de

In [2]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 2
epochs = 60
(ds_train, ds_val, ds_test, count_classes) = ir.readData("../FairFace", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(ds_train)

base_model = bm.build_model()

utils.train_model(base_model, epochs, ds_train, ds_val, class_weight)

utils.saveModel(base_model, "base_model")

Delete old data folder: ../museumFaces_split


Copying files: 4897 files [00:00, 10643.49 files/s]


Found 3916 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.


2022-11-23 19:54:57.068083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 19:54:57.068110: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 19:54:57.068137: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (christoph-XPS-13-9370): /proc/driver/nvidia/version does not exist
2022-11-23 19:54:57.068359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Count classes: (2052, 1864, 256, 233, 258, 234)
Weight for class 0: 0.95
Weight for class 1: 1.05
sc
Epoch 1/60


ValueError: When eager execution is enabled, use_resource cannot be set to false.

In [3]:
utils.saveModel(base_model, "base_model.h5")

In [7]:
model = utils.loadModel("base_model.h5")

In [10]:
tl.freezeAllLayers(model)
tl.unfreezeLayers(model, -5)
#for l in model.layers:
#    print(l.name, l.trainable)
model.compile(loss="binary_crossentropy")

(ds_train, ds_val, ds_test, count_classes) = ir.readData("../museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(ds_train)

epochs = 20
utils.train_model(model, epochs, ds_train, ds_val, class_weight)

input_1 False
rescaling False
stem_conv False
stem_bn False
stem_activation False
block1a_project_conv False
block1a_project_bn False
block1a_project_activation False
block1a_add False
block1b_project_conv False
block1b_project_bn False
block1b_project_activation False
block1b_drop False
block1b_add False
block1c_project_conv False
block1c_project_bn False
block1c_project_activation False
block1c_drop False
block1c_add False
block1d_project_conv False
block1d_project_bn False
block1d_project_activation False
block1d_drop False
block1d_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_project_conv False
block2b_project_bn False
block2b_drop False
block2b_add False
block2c_expand_conv False
block2c_expand_bn False
block2c_expand_activation False
block2c_project_conv False
block2c_project_bn False
block2c_dr